In [3]:
import sys
import os
import time
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from pathlib import Path
from torch.utils.data import DataLoader, Subset
import random

# Add project root to sys.path
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.append(project_root)

from src.eval_5shot import run_5shot_eval
    
# Configuration
DATA_DIR = os.path.join(project_root, "data/plankton")
EXPERIMENTS_DIR = os.path.join(project_root, "experiments/plankton")
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
SEED = 42

print(f"Device: {DEVICE}")
print(f"Data Directory: {DATA_DIR}")

/home/ken/Computer-Vision-Final-Project/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Device: cuda
Data Directory: /home/ken/Computer-Vision-Final-Project/data/plankton


## Experiment Runner
A function to run the full evaluation pipeline for a given method.

In [4]:
results = []

def run_experiment(method, model_path=None, lora_rank=8, flylora_k=4, dataset_name="Plankton"):
    acc, training_time = run_5shot_eval(
        data_dir=DATA_DIR,
        method=method,
        model_path=model_path,
        lora_rank=lora_rank,
        flylora_k=flylora_k,
        seed=SEED,
        batch_size=60
    )
    
    results.append({
        "Dataset": dataset_name,
        "Method": method,
        "Training Time (s)": round(training_time, 2),
        "Accuracy (%)": round(acc, 2)
    })
    return acc

print("Function `run_experiment` defined successfully!")

Function `run_experiment` defined successfully!


## 1. Baseline (Original BioCLIP)

In [5]:
run_experiment("linear_probe", model_path=None)


=== Running 5-Shot Evaluation: linear_probe ===
Initializing BioCLIP backbone...
Loading BioCLIP model: hf-hub:imageomics/bioclip-2...
Updating classifier head for 102 classes...
Using Random 5-Shot Splits (Baseline Protocol)...
Applying Linear Probe (Freezing encoder, unfreezing classifier)...
Using Pre-trained BioCLIP Backbone (Baseline)
Training 5-shot classifier...
Evaluating...
Test Accuracy: 38.65%


38.649189454447004

## 2. LoRA (Fine-Tuned)

In [6]:
lora_path = os.path.join(EXPERIMENTS_DIR, "lora/best_model.pth")
run_experiment("lora", model_path=lora_path, lora_rank=8)


=== Running 5-Shot Evaluation: lora ===
Initializing BioCLIP backbone...
Loading BioCLIP model: hf-hub:imageomics/bioclip-2...
Updating classifier head for 102 classes...
Generating splits to isolate UNSEEN data...
Full Dataset Split: Train=378706, Val=47284, Test=47283
Selecting 5-shot samples from the TEST split only...
5-Shot Split (Unseen): Train=450, Test=46833
Applying LoRA with r=8, alpha=16, dropout=0.1, targets=['c_fc', 'out_proj']...
Loading backbone from /home/ken/Computer-Vision-Final-Project/experiments/plankton/lora/best_model.pth
Training 5-shot classifier...
Evaluating...
Test Accuracy: 82.19%


82.18777357845963

## 3. FlyLoRA (Fine-Tuned)

In [7]:
flylora_path = os.path.join(EXPERIMENTS_DIR, "flylora/best_model.pth")
run_experiment("flylora", model_path=flylora_path, lora_rank=8, flylora_k=4)


=== Running 5-Shot Evaluation: flylora ===
Initializing BioCLIP backbone...
Loading BioCLIP model: hf-hub:imageomics/bioclip-2...
Updating classifier head for 102 classes...
Generating splits to isolate UNSEEN data...
Full Dataset Split: Train=378706, Val=47284, Test=47283
Selecting 5-shot samples from the TEST split only...
5-Shot Split (Unseen): Train=450, Test=46833
Applying FlyLoRA with r=8, k=4, dropout=0.1, targets=['c_fc', 'out_proj']...
Replacing model.visual.transformer.resblocks.0.attn.out_proj with FlyLoRA
Replacing model.visual.transformer.resblocks.0.mlp.c_fc with FlyLoRA
Replacing model.visual.transformer.resblocks.1.attn.out_proj with FlyLoRA
Replacing model.visual.transformer.resblocks.1.mlp.c_fc with FlyLoRA
Replacing model.visual.transformer.resblocks.2.attn.out_proj with FlyLoRA
Replacing model.visual.transformer.resblocks.2.mlp.c_fc with FlyLoRA
Replacing model.visual.transformer.resblocks.3.attn.out_proj with FlyLoRA
Replacing model.visual.transformer.resblocks.3.

80.69096577199838

## 4. DoRA (Fine-Tuned)

In [8]:
dora_path = os.path.join(EXPERIMENTS_DIR, "dora/best_model.pth")
run_experiment("dora", model_path=dora_path, lora_rank=8)


=== Running 5-Shot Evaluation: dora ===
Initializing BioCLIP backbone...
Loading BioCLIP model: hf-hub:imageomics/bioclip-2...
Updating classifier head for 102 classes...
Generating splits to isolate UNSEEN data...
Full Dataset Split: Train=378706, Val=47284, Test=47283
Selecting 5-shot samples from the TEST split only...
5-Shot Split (Unseen): Train=450, Test=46833
Applying DoRA with r=8, alpha=16, dropout=0.1, targets=['c_fc', 'out_proj']...
Loading backbone from /home/ken/Computer-Vision-Final-Project/experiments/plankton/dora/best_model.pth
Training 5-shot classifier...
Evaluating...
Test Accuracy: 82.84%


82.83688851877949

## 5. PiSSA (Fine-Tuned)

In [9]:
pissa_path = os.path.join(EXPERIMENTS_DIR, "pissa/best_model.pth")
run_experiment("pissa", model_path=pissa_path, lora_rank=8)


=== Running 5-Shot Evaluation: pissa ===
Initializing BioCLIP backbone...
Loading BioCLIP model: hf-hub:imageomics/bioclip-2...
Updating classifier head for 102 classes...
Generating splits to isolate UNSEEN data...
Full Dataset Split: Train=378706, Val=47284, Test=47283
Selecting 5-shot samples from the TEST split only...
5-Shot Split (Unseen): Train=450, Test=46833
Applying PiSSA with r=8, alpha=16, dropout=0.1, targets=['c_fc', 'out_proj']...
Loading backbone from /home/ken/Computer-Vision-Final-Project/experiments/plankton/pissa/best_model.pth
Training 5-shot classifier...
Evaluating...
Test Accuracy: 81.16%


81.16499049815302

## Summary of Results

In [10]:
df = pd.DataFrame(results)
print(df)

# Save to CSV
df.to_csv("plankton_5shot_results.csv", index=False)
print("Results saved to plankton_5shot_results.csv")

    Dataset        Method  Training Time (s)  Accuracy (%)
0  Plankton  linear_probe               2.32         38.65
1  Plankton          lora               2.29         82.19
2  Plankton       flylora               2.46         80.69
3  Plankton          dora               2.55         82.84
4  Plankton         pissa               2.25         81.16
Results saved to plankton_5shot_results.csv
